<a href="https://colab.research.google.com/github/sakuronohana/cas_datenanalyse/blob/master/Semesterarbeit/CAS_DA_SA_Payment_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img 
src="https://www.ffhs.ch/templates/ffhs/img/logo@2x.png" width="100"> 
###DaAn, Data Analysis, MAS/CAS Web4B 2018, ZH1, FS19, Dr. Tödtli Beat###

##*Semesterarbeit von Patrik Di Lena*##
#Betrugserkennung Zahlungsverkehr#



##Ziel##
Auf der Grundlage von maschinellen Lernalgorithmen soll ein Betrugserkennungssystem für den Zahlungsverkehr einer ausschliesslich in der Schweiz tätigen Bank erstellt werden. Auf Basis historisierter Daten in Bezug auf das E-Banking Session- und Zahlungsverhalten von Bankkunden sollen mögliche Betrugsintensionen erkannt und für bestimmte Risikogruppen vorhergesagt werden.

##Aufgabenstellung##

Aus dem oben formulierten Ziel ergeben sich zwei wesentliche Aufgabenstellungen:
1.	Erkennung betrügerischer Handlungen im Zahlungsverkehr
2.	Ermittlung potenzieller Kundenrisikogruppen

##Betrugsanfällige Risikogruppen##
Die für den Betrug im Zahlungsverkehr anfälligen Gruppen werden auf Basis der folgenden Merkmale durch den Lernalgorithmus bestimmt:

*	Geschlecht
*	Alter
*	Zivilstand
*	Nationalität
*	Kontosaldo
*	Wohnort
*	Region
*	Zahlungsmethode (E-Rechnung, Einzahlungsschein etc.)
*	Rechtsform (Juristisch oder natürliche Person)

## Datensatz##
Die im Rahmen dieser Semesterarbeit verwendeten Daten entstammen, von ihrer Datenstruktur her, aus einem bereits eingesetzten nicht ML-basierten E-Banking Betrugserkennungssystem. Die Datenwerte wurde zu Wahrung der datenschutzrechtlichen und bankengesetzlichen Aspekte mit Hilfe des folgenden Python-Scripts künstlich erzeugt und beinhalten somit keine reale Zahlungs-, Kunden-, und betrugsspezifischen Informationen. Weiter wurden einige durch die Bank definierten Spezifikationen bzw. Merkmale aus sicherheitstechnischen Gründen umbenannt.  

Datenselektion:

- 5'000 synthetische Kunden (Vorname, Name, Strasse, Ort, Kanton, Sprachregion, Geschlecht, Alter, Zivilstand,  Nationalität, Kontonummer, Vertragsnummer, Rechtsform )

- 100'000 synthetische Zahlungen (Transaktionsid, ,Erstellungdatum/Zeit, Empfängerkonto, Empfängerbank, Empfängerland, Währung, Betrag, Valuta Datum)








###Datengeneratoren###

Während die Stammdaten hauptsächlich im Excel manuell von verschiedenen Quellen 

Datenquellen:

Open Government Data https://opendata.swiss/de/


In [35]:
# IBAN Generator

import numpy as np
import pandas as pd


iban_country_code = 'CH53'
iban_bank_code = '0077'
 
iban_cc = pd.read_csv('https://raw.githubusercontent.com/sakuronohana/cas_datenanalyse/master/Semesterarbeit/Dataset/iban_list.csv', delimiter=';')
iban_cc.columns=['Country','Code','Checkdig','Length','Sample']

cc_ch = iban_cc['Country'].where(iban_cc['Country'] != 'Albanien')


iban_account1 = np.int64(np.random.randint(2000,3000,20800))
iban_account2 = np.int64(np.random.randint(3000,4000,20800))
iban_account3 = np.int64(np.random.randint(5000,6000,20800))
iban_account4 = np.random.randint(1,9,20800)


iban = {'a':iban_account1,'b':iban_account2 ,'c':iban_account3 ,'d':iban_account4}
iban = pd.DataFrame(iban)

ccode = []
bcode = []

for row in range(len(iban)):
  if row > 1:
    ccode.append(iban_country_code)
    bcode.append(iban_bank_code)
   
cc = pd.DataFrame(ccode)
bc = pd.DataFrame(bcode)
iban.insert(0,'cc',cc)
iban.insert(1,'bc',bc)

iban.to_csv('gen_iban.csv',index=False, sep=' ')

data = pd.read_csv('gen_iban.csv',header=None)

print(data)

                                0
0                   cc bc a b c d
1      CH53 0077 2858 3890 5011 6
2      CH53 0077 2302 3261 5820 7
3      CH53 0077 2011 3771 5325 2
4      CH53 0077 2555 3309 5895 3
5      CH53 0077 2025 3414 5424 5
6      CH53 0077 2511 3954 5979 3
7      CH53 0077 2710 3140 5367 7
8      CH53 0077 2048 3405 5755 1
9      CH53 0077 2587 3403 5527 3
10     CH53 0077 2221 3091 5378 6
11     CH53 0077 2886 3533 5368 7
12     CH53 0077 2318 3136 5268 7
13     CH53 0077 2073 3229 5830 7
14     CH53 0077 2264 3641 5106 7
15     CH53 0077 2988 3787 5718 3
16     CH53 0077 2149 3483 5599 3
17     CH53 0077 2003 3347 5257 6
18     CH53 0077 2364 3774 5678 3
19     CH53 0077 2315 3569 5404 5
20     CH53 0077 2729 3889 5621 4
21     CH53 0077 2182 3607 5025 7
22     CH53 0077 2877 3188 5043 6
23     CH53 0077 2557 3942 5756 5
24     CH53 0077 2744 3741 5472 3
25     CH53 0077 2446 3464 5059 8
26     CH53 0077 2665 3882 5361 1
27     CH53 0077 2621 3222 5031 8
28     CH53 00

In [0]:
# Payment Transaction Generator




